In [1]:
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs4
import re
import json
import requests
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
import fantasy_utils as fu

In [2]:
from fantasy_utils import Fantasy_Projections
fp = Fantasy_Projections()

In [3]:
# Connect to database
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
#cur = engine.cursor()
fu.show_all_tables()

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,roster
8,stats
9,players2023


In [4]:
ids = fp.load_id_map()
ids.columns

Index(['IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME', 'LASTNAME', 'TEAM',
       'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME', 'MLBID', 'MLBNAME',
       'CBSID', 'CBSNAME', 'RETROID', 'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID',
       'ESPNNAME', 'KFFLNAME', 'DAVENPORTID', 'BPID', 'YAHOOID', 'YAHOONAME',
       'MSTRBLLNAME', 'BATS', 'THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST',
       'ROTOWIREID', 'FANDUELNAME', 'FANDUELID', 'DRAFTKINGSNAME', 'OTTONEUID',
       'HQID', 'RAZZBALLNAME', 'FANTRAXID', 'FANTRAXNAME', 'ROTOWIRENAME',
       'ALLPOS', 'NFBCLASTFIRST', 'ACTIVE', 'UNDERDOG', 'RAZZBALLID',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54'],
      dtype='object')

In [5]:
def refresh():
    return pd.read_sql('SELECT * FROM players', engine)

In [6]:
players = refresh()

## Section 1
#### Adding and Editing Players in the Database

In [7]:
players = refresh()
players.columns

Index(['cbsid', 'CBSNAME', 'IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME',
       'LASTNAME', 'TEAM', 'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME',
       'MLBID', 'MLBNAME', 'RETROID', 'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID',
       'ESPNNAME', 'BPID', 'YAHOOID', 'YAHOONAME', 'MSTRBLLNAME', 'BATS',
       'THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST', 'ROTOWIREID', 'FANTRAXID',
       'FANTRAXNAME', 'ROTOWIRENAME', 'ALLPOS', 'NFBCLASTFIRST', 'ACTIVE'],
      dtype='object')

In [8]:
players[players['IDFANGRAPHS']=='35108']

,cbsid,CBSNAME,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,MLBID,MLBNAME,RETROID,BREFID,NFBCID,NFBCNAME,ESPNID,ESPNNAME,BPID,YAHOOID,YAHOONAME,MSTRBLLNAME,BATS,THROWS,FANTPROSNAME,LASTCOMMAFIRST,ROTOWIREID,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE
3889,29426043.0,Cam Smith,smithca01,Cam Smith,2/22/2003,Cam,Smith,HOU,AL,3B,35108,Cam Smith,701358.0,Cam Smith,None,smithca07,12331.0,Cam Smith,NaN,None,NaN,64330.0,Cam Smith,"Smith, Cam",R,R,Cam Smith,"Smith, Cam",17994.0,*060jo*,Cam Smith,Cam Smith,3B,"Smith, Cam",Y


In [11]:
# Players from ids with IDFANGRAPHS merged with players DB 
# Looking to use the CBSID for cbsid
z = ids[ids['IDFANGRAPHS'].notna()][['IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME',
       'LASTNAME', 'TEAM', 'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME',
       'MLBID', 'MLBNAME', 'RETROID', 'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID',
       'ESPNNAME', 'BPID', 'YAHOOID', 'YAHOONAME', 'MSTRBLLNAME', 'BATS',
       'THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST', 'ROTOWIREID', 'FANTRAXID',
       'FANTRAXNAME', 'ROTOWIRENAME', 'ALLPOS', 'NFBCLASTFIRST', 'ACTIVE', 'CBSID', 'CBSNAME']].merge(players[['cbsid', 'CBSNAME', 'IDFANGRAPHS']], on='IDFANGRAPHS', how='left')
z = z[z['cbsid'].isna()].sort_values('CBSID')
z[z['CBSID'].notna()]

,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,MLBID,MLBNAME,RETROID,BREFID,NFBCID,NFBCNAME,ESPNID,ESPNNAME,BPID,YAHOOID,YAHOONAME,MSTRBLLNAME,BATS,THROWS,FANTPROSNAME,LASTCOMMAFIRST,ROTOWIREID,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE,CBSID,CBSNAME_x,cbsid,CBSNAME_y
2170,muncyma02,Max Muncy,8/25/2002,Max,Muncy,OAK,AL,SS,29779,Max Muncy,691777.0,Max Muncy,NaN,muncyma02,12301.0,Max Muncy,NaN,NaN,NaN,NaN,NaN,NaN,R,R,NaN,"Muncy, Max",NaN,*05t9l*,Max Muncy,NaN,SS,"Muncy, Max",Y,2106304.0,Max Muncy,NaN,NaN
157,avilape01,Pedro Avila,1/14/1997,Pedro,Avila,CLE,AL,P,18864,Pedro Avila,658648.0,Pedro Avila,NaN,avilape01,10310.0,Pedro Avila,NaN,Pedro Avila,105525.0,11230.0,Pedro Avila,NaN,R,R,Pedro Avila,"Avila, Pedro",14626.0,*03to4*,Pedro Avila,Pedro Avila,RP,"Avila, Pedro",Y,2507525.0,Pedro Avila,NaN,Pedro Avila
403,brooktr01,Trenton Brooks,7/3/1995,Trenton,Brooks,SD,NL,1B,19384,Trenton Brooks,670092.0,Trenton Brooks,NaN,brooktr01,12095.0,Trenton Brooks,NaN,Trenton Brooks,107537.0,NaN,Trenton Brooks,"Brooks, Trenton",L,L,Trenton Brooks,"Brooks, Trenton",18681.0,*0420f*,Trenton Brooks,Trenton Brooks,1B,"Brooks, Trenton",Y,2804230.0,Trenton Brooks,NaN,NaN
1286,hamilia01,Ian Hamilton,6/16/1995,Ian,Hamilton,NYY,AL,P,19261,Ian Hamilton,641656.0,Ian Hamilton,NaN,hamilia01,10087.0,Ian Hamilton,41127.0,Ian Hamilton,107918.0,11142.0,Ian Hamilton,NaN,R,R,Ian Hamilton,"Hamilton, Ian",15728.0,*041w8*,Ian Hamilton,Ian Hamilton,RP,"Hamilton, Ian",Y,2823219.0,Ian Hamilton,NaN,Ian Hamilton
2139,moretda01,Dauri Moreta,4/15/1996,Dauri,Moreta,PIT,NL,P,21101,Dauri Moreta,664294.0,Dauri Moreta,NaN,moretda01,11294.0,Dauri Moreta,41874.0,Dauri Moreta,106516.0,12274.0,Dauri Moreta,NaN,R,R,Dauri Moreta,"Moreta, Dauri",17632.0,*05tux*,Dauri Moreta,Dauri Moreta,P,"Moreta, Dauri",Y,2824544.0,Dauri Moreta,NaN,Dauri Moreta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,youngbr01,Brandon Young,8/19/1998,Brandon,Young,BAL,AL,P,sa3014926,Brandon Young,687064.0,Brandon Young,NaN,youngbr01,12223.0,Brandon Young,4414339.0,Brandon Young,144696.0,64813.0,Brandon Young,"Young, Brandon",R,R,Brandon Young,"Young, Brandon",16949.0,*05r3a*,Brandon Young,Brandon Young,P,"Young, Brandon",Y,29631474.0,Brandon Young,NaN,NaN
1476,huntkc01,K.C. Hunt,7/14/2000,K.C.,Hunt,MIL,NL,P,sa3022937,K.C. Hunt,687268.0,K.C. Hunt,NaN,huntkc01,12218.0,K.C. Hunt,NaN,K.C. Hunt,144207.0,64900.0,K.C. Hunt,NaN,L,R,K.C. Hunt,"Hunt, K.C.",19765.0,*066b3*,K.C. Hunt,K.C. Hunt,P,"Hunt, K.C.",Y,29632440.0,K.C. Hunt,NaN,NaN
3022,suganto01,Tomoyuki Sugano,10/11/1989,Tomoyuki,Sugano,BAL,AL,P,35321,Tomoyuki Sugano,608372.0,Tomoyuki Sugano,NaN,suganto01,11148.0,Tomoyuki Sugano,NaN,Tomoyuki Sugano,70637.0,64895.0,Tomoyuki Sugano,NaN,R,R,Tomoyuki Sugano,"Sugano, Tomoyuki",14712.0,*04b01*,Tomoyuki Sugano,Tomoyuki Sugano,P,"Sugano, Tomoyuki",Y,29632883.0,Tomoyuki Sugano,NaN,NaN
3449,yohocr01,Craig Yoho,10/23/1999,Craig,Yoho,MIL,NL,P,sa3023032,Craig Yoho,684974.0,Craig Yoho,NaN,yohocr01,12225.0,Craig Yoho,NaN,NaN,NaN,63247.0,NaN,NaN,R,R,NaN,"Yoho, Craig",19413.0,*06fzq*,Craig Yoho,NaN,P,"Yoho, Craig",Y,29658270.0,Craig Yoho,NaN,NaN


In [12]:
# Loops through previous cell
# Inserts players in the z dataframe into the database

with engine.connect() as conn:
    for i, row in z[z['CBSID'].notna()].iterrows():
        print(row.to_dict())
        conn.execute(text("INSERT INTO players ('IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME', 'LASTNAME', 'TEAM', 'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME','MLBID', 'MLBNAME', 'RETROID', \
                        'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID', 'ESPNNAME', 'BPID', 'YAHOOID', 'YAHOONAME', 'MSTRBLLNAME', 'BATS','THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST', 'ROTOWIREID', 'FANTRAXID',\
                        'FANTRAXNAME', 'ROTOWIRENAME', 'ALLPOS', 'NFBCLASTFIRST', 'ACTIVE', 'CBSID', 'CBSNAME') VALUES (:IDPLAYER, :PLAYERNAME, :BIRTHDATE, :FIRSTNAME, :LASTNAME, :TEAM, :LG, :POS, :IDFANGRAPHS, \
                        :FANGRAPHSNAME, :MLBID, :MLBNAME, :RETROID, :BREFID, :NFBCID, :NFBCNAME, :ESPNID, :ESPNNAME, :BPID, :YAHOOID, :YAHOONAME, :MSTRBLLNAME, :BATS, :THROWS, :FANTPROSNAME, :LASTCOMMAFIRST, :ROTOWIREID, \
                        :FANTRAXID, :FANTRAXNAME, :ROTOWIRENAME, :ALLPOS, :NFBCLASTFIRST, :ACTIVE, :CBSID, :CBSNAME_x)"), row.to_dict())
        conn.commit()
        

{'IDPLAYER': 'muncyma02', 'PLAYERNAME': 'Max Muncy', 'BIRTHDATE': '8/25/2002', 'FIRSTNAME': 'Max', 'LASTNAME': 'Muncy', 'TEAM': 'OAK', 'LG': 'AL', 'POS': 'SS', 'IDFANGRAPHS': '29779', 'FANGRAPHSNAME': 'Max Muncy', 'MLBID': 691777.0, 'MLBNAME': 'Max Muncy', 'RETROID': nan, 'BREFID': 'muncyma02', 'NFBCID': 12301.0, 'NFBCNAME': 'Max Muncy', 'ESPNID': nan, 'ESPNNAME': nan, 'BPID': nan, 'YAHOOID': nan, 'YAHOONAME': nan, 'MSTRBLLNAME': nan, 'BATS': 'R', 'THROWS': 'R', 'FANTPROSNAME': nan, 'LASTCOMMAFIRST': 'Muncy, Max', 'ROTOWIREID': nan, 'FANTRAXID': '*05t9l*', 'FANTRAXNAME': 'Max Muncy', 'ROTOWIRENAME': nan, 'ALLPOS': 'SS', 'NFBCLASTFIRST': 'Muncy, Max', 'ACTIVE': 'Y', 'CBSID': 2106304.0, 'CBSNAME_x': 'Max Muncy', 'cbsid': nan, 'CBSNAME_y': nan}
{'IDPLAYER': 'avilape01', 'PLAYERNAME': 'Pedro Avila', 'BIRTHDATE': '1/14/1997', 'FIRSTNAME': 'Pedro', 'LASTNAME': 'Avila', 'TEAM': 'CLE', 'LG': 'AL', 'POS': 'P', 'IDFANGRAPHS': '18864', 'FANGRAPHSNAME': 'Pedro Avila', 'MLBID': 658648.0, 'MLBNAME':

In [ ]:
with engine.connect() as conn:
    for i, row in ids[ids['IDFANGRAPHS']=='sa3012639'].iterrows():
        print(row.to_dict())
        conn.execute(text("INSERT INTO players ('IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME', 'LASTNAME', 'TEAM', 'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME','MLBID', 'MLBNAME', 'RETROID', \
                        'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID', 'ESPNNAME', 'BPID', 'YAHOOID', 'YAHOONAME', 'MSTRBLLNAME', 'BATS','THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST', 'ROTOWIREID', 'FANTRAXID',\
                        'FANTRAXNAME', 'ROTOWIRENAME', 'ALLPOS', 'NFBCLASTFIRST', 'ACTIVE', 'CBSID', 'CBSNAME') VALUES (:IDPLAYER, :PLAYERNAME, :BIRTHDATE, :FIRSTNAME, :LASTNAME, :TEAM, :LG, :POS, :IDFANGRAPHS, \
                        :FANGRAPHSNAME, :MLBID, :MLBNAME, :RETROID, :BREFID, :NFBCID, :NFBCNAME, :ESPNID, :ESPNNAME, :BPID, :YAHOOID, :YAHOONAME, :MSTRBLLNAME, :BATS, :THROWS, :FANTPROSNAME, :LASTCOMMAFIRST, :ROTOWIREID, \
                        :FANTRAXID, :FANTRAXNAME, :ROTOWIRENAME, :ALLPOS, :NFBCLASTFIRST, :ACTIVE, :CBSID, :CBSNAME)"), row.to_dict())
        conn.commit()

In [13]:
players = refresh()

In [14]:
# Looks for players with more than one entry in the database. 
# Uses name b/c some players don't have cbsid. 
# If more than 1 exists, check DOB, should be more than 1 to indicate actually different players
# Check IDFANGRAPHS for those starting with 'sa' which indicate minors. If player has a MLB version of the ID, then the minors entry can be deleted
players.groupby('CBSNAME').agg({'cbsid':'first', 'BIRTHDATE':'unique', 'IDFANGRAPHS':'unique', 'TEAM':'count'}).sort_values('TEAM').query('TEAM>1')

,cbsid,BIRTHDATE,IDFANGRAPHS,TEAM
CBSNAME,,,,
Forrest Wall,2135267.0,[11/20/1995],"[sa828682, 16496]",2
Forrest Whitley,2250602.0,[9/15/1997],"[sa917930, 19586]",2
Garrett Acton,29084569.0,"[None, 6/15/1998]","[None, 27583]",2
Franklyn Kilome,2167487.0,[6/25/1995],"[sa830592, 17861]",2
Gordon Graceffo,28862895.0,[3/17/2000],"[sa3016849, 29519]",2
...,...,...,...,...
Will Smith,2250617.0,"[3/28/1995, 7/10/1989]","[19197, 8048]",2
Brant Hurter,29075483.0,"[None, 9/6/1998]","[None, 30133]",2
Bryan Hoeing,28862328.0,"[None, 10/19/1996]","[None, 26304]",2


In [9]:
pd.read_sql("SELECT * FROM players WHERE CBSNAME='Chandler Simpson'", engine)

,cbsid,CBSNAME,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,MLBID,MLBNAME,RETROID,BREFID,NFBCID,NFBCNAME,ESPNID,ESPNNAME,BPID,YAHOOID,YAHOONAME,MSTRBLLNAME,BATS,THROWS,FANTPROSNAME,LASTCOMMAFIRST,ROTOWIREID,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE
0,29075503.0,Chandler Simpson,simpsch01,Chandler Simpson,11/18/2000,Chandler,Simpson,TB,AL,OF,sa3020219,Chandler Simpson,802415.0,Chandler Simpson,None,simpsch01,12212.0,Chandler Simpson,None,Chandler Simpson,162174.0,None,Chandler Simpson,None,L,R,Chandler Simpson,"Simpson, Chandler",18046.0,*0622w*,Chandler Simpson,Chandler Simpson,OF,"Simpson, Chandler",Y


In [ ]:
with engine.connect() as conn:
    conn.execute(text("DELETE FROM players WHERE CBSNAME='Dane Myers' AND MLBID IS NULL"))
    conn.commit()

In [ ]:
with engine.connect() as conn:
    for nm in ['Miguel Vargas', 'Jose Herrera', 'Patrick Bailey', 'Max Muncy',
       'Lawrence Butler', 'Jose Caballero', 'David Hamilton', 'Dairon Blanco',
       'Heston Kjerstad', 'Davis Schneider', 'Richie Palacios',
       'Blake Perkins', 'Tyler Fitzgerald', 'Jacob Young']:
        conn.execute(text("DELETE FROM players WHERE BIRTHDATE IS NULL AND CBSNAME=:nm"), {'nm':nm})
        conn.commit()

In [ ]:
# Looking for players without cbsid and ids that do have cbsid. If both have IDFANGRAPHS, then join and use to apply the cbsid
x = players[players['cbsid'].isna()].merge(ids[ids['CBSID'].notna()][['CBSID', 'IDFANGRAPHS']], on='IDFANGRAPHS', how='inner')
x.fillna({'cbsid':x['CBSID']}, inplace=True)
x

In [ ]:
# Loop through previous cell's players and update with newly found cbsid

with engine.connect() as conn:
    for i, row in x[x['cbsid'].notna()].sort_values('CBSNAME').iterrows():
        conn.execute(text("UPDATE players SET cbsid=:cbsid WHERE IDFANGRAPHS=:fgid"), {'cbsid':row['CBSID'], 'fgid':row['IDFANGRAPHS']})
        conn.commit()

In [ ]:
temp = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-cbs-proj-p.csv')

with engine.connect() as conn:
    for i, row in temp.iterrows():
        if row['cbsid'] not in players.cbsid.tolist():
            print(row[['cbsid', 'CBSNAME', 'Team']].to_dict())
            try:
                
                conn.execute(
                    text("INSERT INTO players (PLAYERNAME, TEAM, CBSNAME, CBSID) VALUES (:CBSNAME, :Team, :CBSNAME, :cbsid)"),
                    row.to_dict()
                )
                conn.commit()
                
            except Exception as e:
                print(f"Error inserting row {row.to_dict()}: {e}")


In [ ]:
with engine.connect() as conn:
    for i, row in players.groupby('CBSNAME').agg({'cbsid':'first', 'BIRTHDATE':'unique', 'IDFANGRAPHS':'unique', 'TEAM':'count'}).sort_values('TEAM').query('TEAM>1').reset_index().iterrows():
        if len(row['BIRTHDATE']) == 1:
            if row['BIRTHDATE'] == [None]:
                print(i, row['CBSNAME'], row['BIRTHDATE'], row['cbsid'])
                conn.execute(text("DELETE FROM players WHERE BIRTHDATE IS NULL AND cbsid=:cbsid"),{'cbsid':row['cbsid']})
                conn.commit()

In [ ]:
steamer = pd.read_csv(rf"C:\Users\pddnh\Documents\GitHub\xdl\data\2025-steamer-proj-h.csv")

In [ ]:
with engine.connect() as conn:
    for i, row in h_without_cbsid[['Name', 'playerid']].drop_duplicates().iterrows():
        t = conn.execute(text("SELECT CBSNAME, cbsid, IDFANGRAPHS, FANGRAPHSNAME FROM players WHERE CBSNAME=:name"), {'name':row['Name']})
        for val in t:
            print(val)
        #conn.execute(text("UPDATE players SET IDFANGRAPHS=:fgid, FANGRAPHSNAME=:name WHERE CBSNAME=:name"), {'fgid':row['playerid'], 'name':row['Name']})
        #conn.commit()
    

In [10]:
players = refresh()
nm = 'Chandler Simpson'
players.query('CBSNAME==@nm')[['IDFANGRAPHS', 'MLBID', 'PLAYERNAME', 'cbsid', 'CBSNAME']]

,IDFANGRAPHS,MLBID,PLAYERNAME,cbsid,CBSNAME
3662,sa3020219,802415.0,Chandler Simpson,29075503.0,Chandler Simpson


In [ ]:
# Manually assign owner during draft

cbsid = 1894627
owner = 'Lima Time'
price = 35
supp = 0

with engine.connect() as conn:
    conn.execute(text(f"UPDATE players2025 SET Owner='{owner}', Paid={price}, Timestamp={datetime.now()}, Supp={supp} WHERE cbsid = 1894627"))
    conn.commit()

In [11]:
# Individually update a player

with engine.connect() as conn:
    conn.execute(text(f"UPDATE players SET IDFANGRAPHS='31912' WHERE IDFANGRAPHS='sa3020219' AND CBSNAME='{nm}'"))
    conn.commit()

In [60]:
# Individually update a player
nm = ' Lindor'
with engine.connect() as conn:
    conn.execute(text(f"UPDATE players SET MLBID=683003 WHERE IDFANGRAPHS = '27863' AND CBSNAME='{nm}'"))
    conn.commit()

In [64]:
with engine.connect() as conn:
    conn.execute(text(f"DELETE FROM players WHERE cbsid='26615394' AND MLBID IS NULL"))
    conn.commit()

In [12]:
players = refresh()
players[players['PLAYERNAME']==nm]

,cbsid,CBSNAME,IDPLAYER,PLAYERNAME,BIRTHDATE,FIRSTNAME,LASTNAME,TEAM,LG,POS,IDFANGRAPHS,FANGRAPHSNAME,MLBID,MLBNAME,RETROID,BREFID,NFBCID,NFBCNAME,ESPNID,ESPNNAME,BPID,YAHOOID,YAHOONAME,MSTRBLLNAME,BATS,THROWS,FANTPROSNAME,LASTCOMMAFIRST,ROTOWIREID,FANTRAXID,FANTRAXNAME,ROTOWIRENAME,ALLPOS,NFBCLASTFIRST,ACTIVE
3662,29075503.0,Chandler Simpson,simpsch01,Chandler Simpson,11/18/2000,Chandler,Simpson,TB,AL,OF,31912,Chandler Simpson,802415.0,Chandler Simpson,None,simpsch01,12212.0,Chandler Simpson,NaN,Chandler Simpson,162174.0,NaN,Chandler Simpson,None,L,R,Chandler Simpson,"Simpson, Chandler",18046.0,*0622w*,Chandler Simpson,Chandler Simpson,OF,"Simpson, Chandler",Y


## Section 2
#### Refactoring code for Fantasy_Projections class

In [ ]:
proj_systems = ['zips', 'zipsdc', 'oopsy', 'steamer', 'fangraphsdc', 'atc', 'thebat', 'thebatx']

In [ ]:
# Loop through possible projection csv files. Load, if they exist, into a list and then concatenate into a dataframe
h = pd.DataFrame()
for proj_file in proj_systems:
    try:
        temp = pd.read_csv(f'data/{datetime.now().year}-{proj_file}-proj-h.csv', encoding="latin-1")
        temp.rename(columns={'SO':'K'},inplace=True)
        temp['sys'] = proj_file
        h = pd.concat([h, temp])
        print(f'Found {proj_file}')
    except:
        print(f"Did not find file {proj_file}")
        pass

h = h.drop(columns='ï»¿Name')
h.rename(columns={'NameASCII':'Name', 'PlayerId':'playerid'},inplace=True)

In [ ]:
h = h.merge(players[['cbsid', 'CBSNAME', 'IDFANGRAPHS']], left_on='playerid', right_on='IDFANGRAPHS', how='left').sort_values('PA', ascending=False)
s = h.groupby('Name')['sys'].count().sort_values()>3
s[s]

In [ ]:
h[h['cbsid'].isna()].shape

In [ ]:
# Add in cbs projections
temp = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-cbs-proj-h.csv')
print(temp.shape)
temp['sys'] = 'cbs'
temp.rename(columns={'Positions':'Pos'},inplace=True)
temp = temp.merge(players[['cbsid', 'IDFANGRAPHS']], on='cbsid', how='left')

h = pd.concat([h,temp]).copy()
h['TB'] = h['1B'] + (h['2B']*2) + (h['3B']*3) + (h['HR']*4)

In [ ]:
h.fillna({'CBS':0, 'playerid':h['IDFANGRAPHS'], 'Name':h['CBSNAME']},inplace=True)

In [ ]:
h[h['cbsid'].isna()].shape

In [ ]:
h.drop(columns=['InterSD', 'InterSK', 'IntraSD'],inplace=True)

In [ ]:
h[h['cbsid'].isna()].shape

In [ ]:
h = h[(h['AB']>1)]
h.shape

In [ ]:
h_without_cbsid = h[h['cbsid'].isna()]
h_without_cbsid.shape

In [ ]:
h = h[h['cbsid'].notna()]
h.shape

In [ ]:
proj = pd.pivot_table(h, index='cbsid', values=['G', 'PA', 'AB', 'H', 'HR', 'R', 'RBI', 'SB', 'BB', 'HBP', 'SF', 'SH', 'TB'], aggfunc='mean')\
    .merge(h[['cbsid', 'playerid', 'Name', 'Team']], on='cbsid', how='inner').drop_duplicates('cbsid') # drop by cbsid b/c when this is the same, the data avgs for the entries are the same
proj['sorter'] = proj['HR']+proj['R']+proj['RBI']+proj['H']+proj['SB']
proj['BA'] = proj['H']/proj['AB']
proj['OBP'] = (proj['H']+proj['BB']+proj['HBP']) / (proj['AB']+proj['BB']+proj['HBP']+proj['SF'])
proj['SLG'] = proj['TB'] / proj['AB']
proj['OPS'] = proj['OBP'] + proj['SLG']
proj.shape

In [ ]:
proj = proj.merge(temp[['cbsid', 'Pos', 'Rank']], how='inner', on='cbsid').drop_duplicates('cbsid')
proj.shape

In [ ]:
# Merge data with CBS auction values
cbs_auction_values = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-cbs-auction-values.csv')[['cbsid', 'CBSNAME', 'CBS']]
proj = proj.merge(cbs_auction_values[['cbsid', 'CBS']], on='cbsid', how='left')
proj.fillna({'CBS':0},inplace=True)
proj.shape

In [ ]:
# Merge data with Fangraphs Auction Calculator values
proj = proj.merge(pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-fangraphs-auction-calculator-h.csv')[['Dollars', 'PlayerId']], left_on='playerid', right_on='PlayerId', how='left')
proj.fillna({'Dollars':0, 'ADP':0},inplace=True)
proj.drop(columns='PlayerId',inplace=True)
proj = proj.sort_values('sorter', ascending=False)
proj.shape

In [ ]:
proj['Primary_Pos'] = proj['Pos'].apply(lambda x: fp.find_primary_pos(x) if type(x) != float else x)

In [ ]:
proj

## Section 3
#### Pitching Projections

In [ ]:
players = refresh()
proj_systems = ['zips', 'zipsdc', 'oopsy', 'steamer', 'fangraphsdc', 'atc', 'thebat', 'thebatx']

In [ ]:
# Loop through possible projection csv files. Load, if they exist, into a list and then concatenate into a dataframe
p = pd.DataFrame()
for proj_file in proj_systems:
    try:
        temp = pd.read_csv(f'data/{datetime.now().year}-{proj_file}-proj-p.csv', encoding="latin-1")
        temp.rename(columns={'SO':'K'},inplace=True)
        temp['sys'] = proj_file
        p = pd.concat([p, temp])
        print(f'Found {proj_file}')
    except:
        print(f"Did not find file {proj_file}")
        pass

p = p.drop(columns='ï»¿Name')
p.rename(columns={'NameASCII':'Name', 'PlayerId':'playerid'},inplace=True)

In [ ]:
p = p.merge(players[['cbsid', 'CBSNAME', 'IDFANGRAPHS']], left_on='playerid', right_on='IDFANGRAPHS', how='left').sort_values('IP', ascending=False)
s = p.groupby('Name')['sys'].count().sort_values()>3
s[s]

In [ ]:
p[p['cbsid'].isna()].shape

In [ ]:
p[p['CBSNAME']=='Braxton Garrett'][['cbsid', 'playerid', 'Name', 'IP', 'ERA', 'ER']]

In [ ]:
temp[temp['cbsid']==2250592]

In [ ]:
# Add in cbs projections
temp = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-cbs-proj-p.csv')
print(temp.shape)
temp['sys'] = 'cbs'
temp.rename(columns={'Positions':'Pos', 'INNs':'IP', 'S':'SV', 'HD':'HLD'},inplace=True)
temp['ER'] = temp['ERA']/9*temp['IP']
temp = temp.merge(players[['cbsid', 'IDFANGRAPHS']], on='cbsid', how='left')

p = pd.concat([p,temp]).copy()

In [ ]:
p = p.drop_duplicates()
p.shape

In [ ]:
p.fillna({'CBS':0, 'playerid':p['IDFANGRAPHS'], 'Name':p['CBSNAME']},inplace=True)
p.drop(columns=['InterSD', 'InterSK', 'IntraSD'],inplace=True)
p.rename(columns={'H':'HA', 'K':'SO'},inplace=True)
p['SvHld'] = p['SV']+p['HLD']

In [ ]:
p[p['cbsid'].isna()].shape

In [ ]:
p = p[(p['IP']>1)]
p.shape

In [ ]:
proj = pd.pivot_table(p, index='cbsid', values=['GS', 'G', 'IP', 'ER', 'HA', 'SO', 'BB', 'W', 'SV', 'HLD', 'SvHld'], aggfunc='mean')\
    .merge(p[['cbsid', 'playerid', 'Name', 'Team']], on='cbsid', how='inner').drop_duplicates('cbsid') # drop by cbsid b/c when this is the same, the data avgs for the entries are the same
proj['sorter'] = proj['SO']+(proj['SvHld']*4)+proj['W']
for i in ['IP', 'GS', 'G', 'HA', 'SO', 'ER', 'BB', 'W', 'SV', 'HLD', 'SvHld']:
    proj.fillna({i:0},inplace=True)
    proj[i] = proj[i].apply(lambda x: int(x))
proj['ERA'] = proj['ER']/proj['IP']*9
proj['WHIP'] = (proj['HA']+proj['BB'])/proj['IP']

proj.shape

In [ ]:
# Merge data with CBS auction values
cbs_auction_values = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-cbs-auction-values.csv')[['cbsid', 'CBSNAME', 'CBS']]
proj = proj.merge(cbs_auction_values[['cbsid', 'CBS']], on='cbsid', how='left')
proj.fillna({'CBS':0},inplace=True)
proj.shape

In [ ]:
# Merge data with Fangraphs Auction Calculator values
proj = proj.merge(pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year}-fangraphs-auction-calculator-p.csv')[['Dollars', 'PlayerId']], left_on='playerid', right_on='PlayerId', how='left')
proj.fillna({'Dollars':0, 'ADP':0},inplace=True)
proj.drop(columns='PlayerId',inplace=True)
proj = proj.sort_values('sorter', ascending=False)
proj.shape

In [ ]:
proj.sort_values('IP')

In [ ]:
proj = proj.merge(temp[['cbsid', 'Pos', 'Rank']], how='left', on='cbsid').drop_duplicates('cbsid')

In [ ]:
proj.loc[(proj['Pos'].isna()) & (proj['GS']>0), 'Pos'] = 'SP'
proj.fillna({'Pos':'RP'},inplace=True)
proj['Primary_Pos'] = proj['Pos'].apply(lambda x: 'SP' if 'SP' in x else 'RP')

In [ ]:
proj[proj['ER']==0].sort_values("IP")

In [ ]:
proj[proj['Name']=='Braxton Garrett']

## Section 4
#### Previous year final stats

In [ ]:
h = pd.DataFrame()
for yr in range(datetime.now().year -3, datetime.now().year):
    temp = pd.read_csv(rf"C:\Users\pddnh\Documents\GitHub\xdl\data\{yr}-final-stats-h.csv")
    h = pd.concat([h,temp])

In [ ]:
h.sort_values('Rank', ascending=True).head(20)

In [ ]:
p = pd.DataFrame()
for yr in range(datetime.now().year -3, datetime.now().year):
    temp = pd.read_csv(rf"C:\Users\pddnh\Documents\GitHub\xdl\data\{yr}-final-stats-p.csv")
    p = pd.concat([p,temp])

In [ ]:
p

## Section 5
#### StatCast 

In [ ]:
h_url = 'https://baseballsavant.mlb.com/leaderboard/custom?year=2024%2C2023&type=batter&filter=&min=25&selections=player_age%2Cab%2Cpa%2Chit%2Chome_run%2Cbabip%2Cb_rbi%2Cr_total_stolen_base%2Cr_run%2Cxba%2Cxslg%2Cwoba%2Cxwoba%2Cavg_swing_speed%2Csquared_up_swing%2Cexit_velocity_avg%2Claunch_angle_avg%2Csweet_spot_percent%2Cbarrel_batted_rate%2Cwhiff_percent%2Cswing_percent%2Csprint_speed&chart=false&x=pa&y=pa&r=no&chartType=beeswarm&sort=xwoba&sortDir=desc'
p_url = 'https://baseballsavant.mlb.com/leaderboard/custom?year=2024%2C2023&type=pitcher&filter=&min=25&selections=player_age%2Cp_formatted_ip%2Cpa%2Cab%2Chit%2Cstrikeout%2Cp_earned_run%2Cp_save%2Cp_out%2Cp_quality_start%2Cp_hold%2Cp_starting_p%2Cxba%2Cxslg%2Cwoba%2Cxwoba%2Cexit_velocity_avg%2Claunch_angle_avg%2Csweet_spot_percent%2Cbarrel_batted_rate%2Cwhiff_percent%2Cff_avg_speed%2Cff_avg_break_z_induced&chart=false&x=pa&y=pa&r=no&chartType=beeswarm&sort=xwoba&sortDir=desc'

stat_cast = pd.DataFrame()

for url in [h_url, p_url]:
    r = requests.get(url)
    
    if r.status_code == 200:
        soup = bs4(r.text, 'html.parser')
    
        # Extract all script tags
        scripts = soup.find_all("script")
        
        for script in scripts:
            if script.string and "var data =" in script.string:
                match = re.search(r'var data = (\[.*?\]);', script.string, re.DOTALL)
                if match:
                    json_str = match.group(1)
                    data = json.loads(json_str)
                    break
        
        sc = pd.DataFrame(data).sort_values('xwoba', ascending=False).reset_index(drop=True)
        sc.rename(columns={'player_id':'MLBID'},inplace=True)
        stat_cast = pd.concat([stat_cast, sc])
    else:
        print('request was unsuccessful')

stat_cast = stat_cast.merge(players[['cbsid', 'MLBID']], on='MLBID', how='left')
stat_cast = stat_cast.merge(ids[['MLBID', 'CBSID']], on='MLBID', how='left')
stat_cast.fillna({'cbsid':stat_cast['CBSID']},inplace=True)
stat_cast.drop(columns='CBSID', inplace=True)
stat_cast['woba_diff'] = stat_cast['woba'].astype(float) - stat_cast['xwoba'].astype(float)
stat_cast['K/9'] = stat_cast['p_strikeout'].astype(float) / (stat_cast['p_out'].astype(float) / 3) * 9
stat_cast['K%'] = stat_cast['p_strikeout'].astype(float) / stat_cast['p_total_pa'].astype(float)
stat_cast['BB%'] = stat_cast['p_walk'].astype(float) / stat_cast['p_total_pa'].astype(float)
stat_cast['K-BB%'] = stat_cast['K%'] - stat_cast['BB%']
stat_cast['player_age'] = stat_cast['player_age'].astype(float)
stat_cast['p_SvHld'] = stat_cast['p_save'].astype(float) + stat_cast['p_hold'].astype(float)
stat_cast['p_whip'] = (stat_cast['p_total_hits'].astype(float) + stat_cast['p_walk'].astype(float) + stat_cast['p_hit_by_pitch'].astype(float)) / (stat_cast['p_out'].astype(float) / 3)
stat_cast.shape

In [ ]:
stat_cast = stat_cast.drop_duplicates(['cbsid', 'player_name', 'year']).sort_values(['pa','year', 'cbsid'],ascending=[False, False, False])
stat_cast.shape

In [ ]:
stat_cast[stat_cast['player_name'].isin(['Wheeler, Zack', 'Uceta, Edwin', 'Williams, Gavin', 'Smith, Cade'])][['player_name', 'MLBID', 'cbsid', 'year', 'p_out', 'p_era', 'p_whip']]

In [ ]:
# Export as csv
stat_cast.to_csv(r'C:\Users\pddnh\Documents\GitHub\xdl\data\2024-statcast.csv',index=False)

In [ ]:
stat_cast[stat_cast['cbsid'].isna()].shape

In [ ]:
[i for i in sc.columns if 'out' in i]

In [ ]:
sc_cols = ['cbsid', 'MLBID', 'player_name', 'year', 'player_age', 'ab', 'pa', 'hit', 'home_run', 'r_total_stolen_base', 'r_run', 'b_rbi', 'batting_avg', 'xba', 'exit_velocity_avg', 'launch_angle_avg', 
            'sweet_spot_percent', 'barrel_batted_rate', 'whiff_percent', 'sprint_speed', 'woba', 'xwoba',
            'p_out', 'p_strikeout', 'p_quality_start', 'p_save', 'p_hold', 'p_total_hits', 'p_walk', 'p_intent_walk', 'p_total_pa', 'p_earned_run', 'p_blown_save', 'ff_avg_speed', 'ff_avg_spin', 'fs_avg_speed', 
            'si_avg_speed', 'si_avg_spin', 'fc_avg_speed', 'fc_avg_spin', 'sl_avg_speed', 'sl_avg_spin', 'cu_avg_speed', 'cu_avg_spin', 'st_avg_speed', 'st_avg_spin', 'ff_avg_break_z_induced', 'fastball_avg_break_z_induced',
            'fastball_avg_spin', 'breaking_avg_spin', 'offspeed_avg_spin', 'woba_diff', 'K%', 'BB%', 'K-BB%', 'K/9'
          ]

In [ ]:
#stat_cast = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year-1}-statcast.csv')

sc_ly = stat_cast[stat_cast['year']==datetime.now().year-1][['cbsid', 'MLBID', 'player_name', 'year', 'player_age', 'woba', 'xwoba', 'woba_diff', 'xba', 'barrel_batted_rate', 'sprint_speed', 'exit_velocity_avg', 'K/9', 'K-BB%', 'fastball_avg_speed', 'fastball_avg_break_z_induced', 'whiff_percent', 'home_run', 'pa', 'r_total_stolen_base', 'r_run', 'b_rbi', 'batting_avg', 'p_quality_start', 'p_SvHld', 'p_strikeout', 'p_out', 'p_era', 'p_whip']]
sc_ly.rename(columns={col:col+'_ly' for col in sc_ly.columns if col not in ['cbsid', 'MLBID', 'player_name', 'year']},inplace=True)

sc_2ly = stat_cast[stat_cast['year']==datetime.now().year-2][['cbsid', 'MLBID', 'player_name', 'year', 'player_age', 'woba', 'xwoba', 'woba_diff', 'xba', 'barrel_batted_rate', 'sprint_speed', 'exit_velocity_avg', 'K/9', 'K-BB%', 'fastball_avg_speed', 'fastball_avg_break_z_induced', 'whiff_percent', 'home_run', 'pa', 'r_total_stolen_base', 'r_run', 'b_rbi', 'batting_avg', 'p_quality_start', 'p_SvHld', 'p_strikeout', 'p_out', 'p_era', 'p_whip']]
sc_2ly.rename(columns={col:col+'_2ly' for col in sc_2ly.columns if col not in ['cbsid', 'MLBID', 'player_name', 'year']},inplace=True)

In [ ]:
sc_ly[sc_ly['player_name'].isin(['Uceta, Edwin', 'Wheeler, Zack', 'Williams, Gavin'])][['player_name', 'cbsid', 'p_out_ly', 'p_whip_ly', 'p_era_ly']]

In [ ]:
sc_2ly[sc_2ly['player_name'].isin(['Uceta, Edwin', 'Wheeler, Zack', 'Williams, Gavin'])][['player_name', 'cbsid', 'p_out_2ly', 'p_whip_2ly', 'p_era_2ly']]

## Section 6
#### Testing

In [4]:
fp.proj_systems = ['atc']

In [6]:
h = fp.preprocess_hitting_projections()

Found atc
Found cbs
Index(['cbsid', 'AB', 'BB', 'H', 'HBP', 'HR', 'K', 'R', 'RBI', 'SB', 'SF',
       'TB', 'playerid', 'Name', 'Team', 'sorter'],
      dtype='object')


KeyError: 'PA'

In [ ]:
h.head()

In [ ]:
stat_cast = pd.read_csv(rf'C:\Users\pddnh\Documents\GitHub\xdl\data\{datetime.now().year-1}-statcast.csv')
sc_ly = stat_cast[stat_cast['year']==datetime.now().year-1][['cbsid', 'MLBID', 'player_name', 'year', 'player_age', 'woba', 'xwoba', 'woba_diff', 'xba', 'barrel_batted_rate', 'sprint_speed', 'exit_velocity_avg', 'K/9', 'K-BB%', 'ff_avg_speed', 'fastball_avg_speed', 'fastball_avg_break_z_induced', 'whiff_percent', 'home_run', 'pa', 'r_total_stolen_base', 'r_run', 'b_rbi', 'batting_avg', 'p_quality_start', 'p_SvHld', 'p_strikeout', 'p_out', 'p_era', 'p_whip']]
sc_ly.rename(columns={col:col+'_ly' for col in sc_ly.columns if col not in ['cbsid', 'MLBID', 'player_name', 'year']},inplace=True)

sc_2ly = stat_cast[stat_cast['year']==datetime.now().year-2][['cbsid', 'MLBID', 'player_name', 'year', 'player_age', 'woba', 'xwoba', 'woba_diff', 'xba', 'barrel_batted_rate', 'sprint_speed', 'exit_velocity_avg', 'K/9', 'K-BB%', 'ff_avg_speed', 'fastball_avg_speed', 'fastball_avg_break_z_induced', 'whiff_percent', 'home_run', 'pa', 'r_total_stolen_base', 'r_run', 'b_rbi', 'batting_avg', 'p_quality_start', 'p_SvHld', 'p_strikeout', 'p_out', 'p_era', 'p_whip']]
sc_2ly.rename(columns={col:col+'_2ly' for col in sc_2ly.columns if col not in ['cbsid', 'MLBID', 'player_name', 'year']},inplace=True)

In [ ]:
sc_ly[sc_ly['player_name'].isin(['Uceta, Edwin', 'Wheeler, Zack', 'Williams, Gavin'])][['player_name', 'cbsid', 'p_out_ly', 'p_whip_ly']]

In [ ]:
p = fp.preprocess_pitching_projections()

In [ ]:
p.head()

In [ ]:
p[p['ER']==0].sort_values('IP')

In [4]:
# Using QS
fp.proj_systems = ['atc']
df = fp.calc_value()

Found atc
Found atc
Doing the BIGAA calculations...
Making adjustments by position...

Total z: 616.8903736462606

H/P split: 0.7226096624300592
Conversion to $: 5.057624714677556
Adding keepers
Completed applying auction values


In [ ]:
cols = ['cbsid', 'Name', 'Pos', 'Rank', 'CBS', 'Dollars', 'z', 'Value', 'Value_ly', 'HR', 'SB', 'R', 'RBI', 'BA', 'ERA', 'WHIP', 'QS', 'SO', 'SvHld', 'ER', 'zQS']

In [ ]:
pd.read_sql("SELECT * FROM players2025 limit 5", engine)

In [ ]:
df[df['z']>=0].groupby('Primary_Pos')[['QS', 'zQS', 'Value']].describe()

In [ ]:
df[df['z']>=0].Value.hist()

In [ ]:
hcols = ['Name', 'Pos', 'Rank', 'CBS', 'Dollars', 'Value', 'PA', 'HR', 'SB', 'R', 'RBI', 'BA', 'H', 'AB', 'zHR', 'zSB', 'zR', 'zRBI', 'zBA', 'z', 'wRC+', 'K%', 'BB%', 'Vol', 'Skew', 'Dim']
pcols = ['Name', 'Pos', 'Rank', 'CBS', 'Dollars', 'Value', 'IP', 'TBF', 'QS', 'SO', 'SvHld', 'ERA', 'WHIP', 'zQS', 'zSO', 'zSvHld', 'zERA', 'zWHIP', 'z', 'K-BB%', 'K/9', 'Vol', 'Skew', 'Dim']

In [ ]:
df[df['Primary_Pos'].isin(['SP'])][pcols].head(15)

In [ ]:
df[df['Name'].isin(['Cole Ragans', 'Zack Wheeler', 'Edwin Uceta'])][['Name', 'player_age_ly', 'player_age_2ly', 'p_out_ly', 'p_out_2ly', 'p_era_ly', 'p_era_2ly', 'p_whip_ly', 'p_whip_2ly', 'K/9_ly', 'K/9_2ly']]

In [ ]:
1 % 12

In [ ]:
i = 1
prevVal = None
adp = []
for v in df.sort_values('Value', ascending=False)['Value']:
    if prevVal:
        adp.append((v+prevVal)/2)
    if i % 12 == 0:
        prevVal = v
    else:
        prevVal = None
    i += 1

In [ ]:
fu.drop_table('players2025')

In [ ]:
fp.data = df

In [6]:
fp.upload('players2025')

In [5]:
df[df['Owner'].notna()].sort_values('Timestamp', ascending=False)[['Name', 'Owner', 'Timestamp']]

,Name,Owner,Timestamp
921,Brice Turang,Young Guns,2025-03-07 12:48:00
1148,Colton Cowser,Young Guns,2025-03-06 12:48:00
564,William Contreras,Young Guns,2025-03-05 12:48:00
1273,Jackson Holliday,Young Guns,2025-03-04 12:48:00
718,Garrett Mitchell,Roid Ragers,2025-03-03 12:48:00
650,Tanner Houck,Roid Ragers,2025-03-02 12:48:00
806,Michael King,Roid Ragers,2025-03-01 12:48:00
208,Aaron Judge,Roid Ragers,2025-02-28 12:48:00
1090,Colt Keith,Brewbirds,2025-02-27 12:48:00
1188,James Wood,Brewbirds,2025-02-26 12:48:00
